<a href="https://colab.research.google.com/github/jajapuramshivasai/Portfolio_Optimization_QAOA/blob/main/portfolio_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install seaborn

In [2]:
import pandas as pd
import numpy as np
import logging

def load_vanguard_data():
    data_file = "data_assets_dump_partial.xlsx"
    try:
        excel_file = pd.ExcelFile(data_file)
        print("Available sheets:", excel_file.sheet_names)
        df = pd.read_excel(data_file, sheet_name=0)
        print(f"\nDataset shape: {df.shape}")
        print(f"Columns: {list(df.columns)}")
        print(f"\nFirst few rows:")
        print(df.head())
        print(f"\nData types:")
        print(df.dtypes)
        print(f"\nMissing values:")
        print(df.isnull().sum())
        return df
    except Exception as e:
        logging.error(f"Error loading data: {e}")
        raise

def analyze_data_structure(df):
    key_columns = {
        'asset_id': ['assetId', 'asset_id', 'id', 'cusip', 'isin'],
        'price': ['price', 'market_price', 'clean_price', 'dirty_price'],
        'quantity': ['quantity', 'position', 'holdings', 'notional'],
        'benchmark_weight': ['benchmark', 'bench', 'target_weight', 'index_weight'],
        'duration': ['duration', 'oad', 'modified_duration', 'eff_duration'],
        'credit_rating': ['rating', 'credit', 'quality', 'grade'],
        'sector': ['sector', 'industry', 'gics', 'classification']
    }
    found_columns = {}
    for category, possible_names in key_columns.items():
        found = []
        for col in df.columns:
            for possible in possible_names:
                if possible.lower() in col.lower():
                    found.append(col)
        found_columns[category] = list(set(found))
    print("\nIdentified key columns:")
    for category, columns in found_columns.items():
        print(f"  {category}: {columns}")
    return found_columns

def prepare_qubo_data(df, found_columns):
    df_clean = df.copy()
    critical_columns = []
    for category in ['asset_id', 'price']:
        if found_columns[category]:
            critical_columns.extend(found_columns[category][:1])
    if critical_columns:
        df_clean = df_clean.dropna(subset=critical_columns)
        print(f"After removing missing critical data: {len(df_clean)} securities")
    column_mapping = {}
    if found_columns['asset_id']:
        column_mapping[found_columns['asset_id'][0]] = 'asset_id'
    if found_columns['price']:
        column_mapping[found_columns['price'][0]] = 'price'
    if found_columns['quantity']:
        column_mapping[found_columns['quantity'][0]] = 'current_position'
    if found_columns['benchmark_weight']:
        column_mapping[found_columns['benchmark_weight'][0]] = 'benchmark_weight'
    if found_columns['duration']:
        column_mapping[found_columns['duration'][0]] = 'duration'
    df_clean = df_clean.rename(columns=column_mapping)
    print(f"Column mapping applied: {column_mapping}")
    print(f"Final dataset shape: {df_clean.shape}")
    return df_clean

def create_objective_matrix(df, penalty_weights, max_assets):
    n_assets = len(df)
    Q = np.zeros((n_assets, n_assets))
    linear_terms = np.zeros(n_assets)
    if 'benchmark_weight' in df.columns:
        benchmark_weights = df['benchmark_weight'].fillna(0).values
        if benchmark_weights.sum() > 0:
            benchmark_weights = benchmark_weights / benchmark_weights.sum()
        for i in range(n_assets):
            linear_terms[i] += penalty_weights['tracking_error'] * (-2 * benchmark_weights[i])
            Q[i, i] += penalty_weights['tracking_error']
    concentration_penalty = 10.0
    for i in range(n_assets):
        for j in range(n_assets):
            if i != j:
                Q[i, j] += concentration_penalty
    budget_target = max_assets
    for i in range(n_assets):
        linear_terms[i] += penalty_weights['budget'] * (-2 * budget_target)
        for j in range(n_assets):
            Q[i, j] += penalty_weights['budget']
    return Q, linear_terms

def add_risk_constraints(Q, linear_terms, df, penalty_weights, max_assets):
    n_assets = len(df)
    if 'duration' in df.columns:
        durations = df['duration'].fillna(0).values
        target_duration = np.mean(durations[durations > 0])
        if target_duration > 0:
            duration_penalty = penalty_weights['risk_limits'] * 0.1
            for i in range(n_assets):
                duration_dev_i = durations[i] - target_duration
                for j in range(n_assets):
                    duration_dev_j = durations[j] - target_duration
                    Q[i, j] += duration_penalty * duration_dev_i * duration_dev_j
    if 'sector' in df.columns:
        sectors = df['sector'].fillna('Other')
        unique_sectors = sectors.unique()
        sector_limit = 0.3
        max_assets_per_sector = int(max_assets * sector_limit)
        sector_penalty = penalty_weights['risk_limits'] * 0.2
        for sector in unique_sectors:
            sector_assets = df[df['sector'] == sector].index.tolist()
            if len(sector_assets) > max_assets_per_sector:
                for i in sector_assets:
                    linear_terms[i] += sector_penalty * (-2 * max_assets_per_sector)
                    for j in sector_assets:
                        Q[i, j] += sector_penalty
    if 'credit_rating' in df.columns:
        ratings = df['credit_rating'].fillna('NR')
        high_risk_ratings = ['BB', 'B', 'CCC', 'CC', 'C', 'D']
        high_risk_mask = ratings.isin(high_risk_ratings)
        if high_risk_mask.sum() > 0:
            high_risk_penalty = penalty_weights['risk_limits'] * 0.3
            high_risk_indices = df[high_risk_mask].index.tolist()
            max_high_risk = int(max_assets * 0.1)
            for i in high_risk_indices:
                linear_terms[i] += high_risk_penalty * (-2 * max_high_risk)
                for j in high_risk_indices:
                    Q[i, j] += high_risk_penalty
    return Q, linear_terms

def add_practical_constraints(Q, linear_terms, df, penalty_weights, max_assets):
    n_assets = len(df)
    if 'price' in df.columns:
        prices = df['price'].fillna(100).values
        min_position_value = 1000000
        small_position_penalty = penalty_weights['turnover']
        for i in range(n_assets):
            if prices[i] > 0:
                if prices[i] * 1000 < min_position_value:
                    linear_terms[i] += small_position_penalty * 0.1
    missing_price_penalty = penalty_weights['turnover'] * 0.5
    for i in range(n_assets):
        if 'price' in df.columns:
            if pd.isna(df.iloc[i]['price']) or df.iloc[i]['price'] <= 0:
                linear_terms[i] += missing_price_penalty
    return Q, linear_terms

def formulate_complete_qubo(df):
    n_assets = len(df)
    penalty_weights = {
        'tracking_error': 1000.0,
        'risk_limits': 500.0,
        'budget': 2000.0,
        'turnover': 100.0
    }
    max_assets = min(31, n_assets // 4)
    Q, linear_terms = create_objective_matrix(df, penalty_weights, max_assets)
    Q, linear_terms = add_risk_constraints(Q, linear_terms, df, penalty_weights, max_assets)
    Q, linear_terms = add_practical_constraints(Q, linear_terms, df, penalty_weights, max_assets)
    Q = (Q + Q.T) / 2
    return Q, linear_terms, max_assets

def solve_classical_approximation(Q, linear_terms, max_assets):
    n_assets = len(linear_terms)
    asset_scores = linear_terms.copy()
    for i in range(len(asset_scores)):
        asset_scores[i] += Q[i, i]
    sorted_indices = np.argsort(asset_scores)
    selected_assets = sorted_indices[:max_assets]
    solution_greedy = np.zeros(n_assets, dtype=int)
    solution_greedy[selected_assets] = 1
    obj_value_greedy = (
        np.dot(solution_greedy, linear_terms) +
        np.dot(solution_greedy, np.dot(Q, solution_greedy))
    )
    best_solution = None
    best_objective = float('inf')
    n_samples = 1000
    for _ in range(n_samples):
        random_indices = np.random.choice(n_assets, max_assets, replace=False)
        solution = np.zeros(n_assets, dtype=int)
        solution[random_indices] = 1
        obj_value = (
            np.dot(solution, linear_terms) +
            np.dot(solution, np.dot(Q, solution))
        )
        if obj_value < best_objective:
            best_objective = obj_value
            best_solution = solution.copy()
    if best_objective < obj_value_greedy:
        return best_solution, best_objective, "random_sampling"
    else:
        return solution_greedy, obj_value_greedy, "greedy"

def analyze_solution(solution, df):
    selected_indices = np.where(solution == 1)[0]
    selected_df = df.iloc[selected_indices].copy()
    print(f"\nSelected {len(selected_indices)} assets:")
    # Additional summary stats as desired...
    return selected_df

def main():
    df = load_vanguard_data()
    found_columns = analyze_data_structure(df)
    df_clean = prepare_qubo_data(df, found_columns).head(16)   #This is to change number of data points
    Q, linear_terms, max_assets = formulate_complete_qubo(df_clean)
    solution, obj_value, method = solve_classical_approximation(Q, linear_terms, max_assets)
    selected_portfolio = analyze_solution(solution, df_clean)
    print(f"\nSOLUTION FOUND using {method}")
    print(f"Objective value: {obj_value:.2f}")
    return Q, linear_terms, solution, df_clean



In [3]:
Q, linear_terms, solution, df_clean = main()
print(solution)

Available sheets: ['Sheet1']

Dataset shape: (2629, 278)
Columns: ['Unnamed: 0', 'isin', 'ccy', 'assetId', 'strategyName', 'secGroup', 'secType', 'cpn', 'posSource', 'baseCcy', 'krd10y', 'krd15y', 'krd20y', 'krd25y', 'krd30y', 'krd1y', 'krd2y', 'krd3m', 'krd3y', 'krd40y', 'krd50y', 'krd5y', 'krd7y', 'oad', 'oas', 'spreadDur', 'issuerCountry', 'countryRisk', 'issuerId', 'issuerLongName', 'issuerShortName', 'issuerTicker', 'portfolioName', 'fund_posQuantity', 'maturity', 'minTradeIncrement', 'minTradeSize', 'fund_orderStatus', 'fund_enriched.mktValue', 'fund_enriched.notionalMktValue', 'enriched.cashladder_tCashBalance', 'enriched.cashladder_tPlus1CashBalance', 'enriched.cashladder_tPlus2CashBalance', 'enriched.cashladder_tPlus3CashBalance', 'enriched.cashladder_tPlus4CashBalance', 'enriched.cashladder_tPlus5CashBalance', 'enriched.cashladder_tPlus6CashBalance', 'enriched.cashladder_tPlus7CashBalance', 'enriched.cashladder_tPlus30CashBalance', 'enriched.cashladder_tPlus60CashBalance', 'e